Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (20000, 28, 28) (20000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (20000, 28, 28, 1) (20000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 3.12501
Minibatch accuracy: 18.8%
Validation accuracy: 13.3%
Minibatch loss at step 50 : 1.77872
Minibatch accuracy: 31.2%
Validation accuracy: 37.1%
Minibatch loss at step 100 : 0.955917
Minibatch accuracy: 56.2%
Validation accuracy: 68.9%
Minibatch loss at step 150 : 1.02583
Minibatch accuracy: 68.8%
Validation accuracy: 72.0%
Minibatch loss at step 200 : 1.12564
Minibatch accuracy: 62.5%
Validation accuracy: 75.0%
Minibatch loss at step 250 : 0.79238
Minibatch accuracy: 75.0%
Validation accuracy: 76.8%
Minibatch loss at step 300 : 0.939195
Minibatch accuracy: 68.8%
Validation accuracy: 78.5%
Minibatch loss at step 350 : 1.20369
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss at step 400 : 0.843373
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 450 : 0.746775
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 500 : 0.720819
Minibatch accuracy: 75.0%
Validation accuracy: 81

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 784]
[10000, 784]
[18724, 784]


In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 3.93185
Minibatch accuracy: 0.0%
Validation accuracy: 10.1%
Minibatch loss at step 50 : 1.76136
Minibatch accuracy: 50.0%
Validation accuracy: 36.4%
Minibatch loss at step 100 : 0.890591
Minibatch accuracy: 75.0%
Validation accuracy: 63.8%
Minibatch loss at step 150 : 0.951342
Minibatch accuracy: 56.2%
Validation accuracy: 69.2%
Minibatch loss at step 200 : 1.47729
Minibatch accuracy: 62.5%
Validation accuracy: 72.8%
Minibatch loss at step 250 : 0.871616
Minibatch accuracy: 75.0%
Validation accuracy: 75.8%
Minibatch loss at step 300 : 1.05023
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 350 : 0.910843
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 400 : 0.612589
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 450 : 0.736313
Minibatch accuracy: 75.0%
Validation accuracy: 76.4%
Minibatch loss at step 500 : 0.759418
Minibatch accuracy: 75.0%
Validation accuracy: 8

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [78]:
batch_size = 16
patch_size = 5
depth = 32
depth_small = 24
num_hidden = 128
MOVING_AVERAGE_DECAY = 0.995

graph = tf.Graph()

with graph.as_default():
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  reduction_weights2 = tf.Variable(tf.truncated_normal(
      [1, 1, depth, depth_small]))
  reduction_biases2 = tf.Variable(tf.zeros([depth_small]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
  layer1_weight_decay = 0.001
  layer2_weight_decay = 0.005
  layer3_weight_decay = 0.005
  layer4_weight_decay = 0.001

    # Track the moving averages of all trainable variables.
  variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
  variable_averages_op = variable_averages.apply(tf.trainable_variables())

  def getVar(v, averaged):
    if averaged:
      return variable_averages.average(v)
    else:
      return v
  
  # Model.
  def model(data, dropout=False, averaged=False):
    if dropout:
      keep_proba = 0.5
    else:
      keep_proba = 1
    
    w1 = getVar(layer1_weights, averaged)
    b1 = getVar(layer1_biases, averaged)
    w2 = getVar(layer2_weights, averaged)
    b2 = getVar(layer2_biases, averaged)
    wrb = getVar(reduction_weights2, averaged)
    brb = getVar(reduction_biases2, averaged)
    w3 = getVar(layer3_weights, averaged)
    b3 = getVar(layer3_biases, averaged)
    w4 = getVar(layer4_weights, averaged)
    b4 = getVar(layer4_biases, averaged)
    
    conv = tf.nn.conv2d(data, w1, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + b1)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, w2, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + b2)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
    
    # reduce = tf.nn.relu(tf.nn.conv2d(pool, wrb, [1, 1, 1, 1], padding='SAME') + brb)
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, w3) + b3)
    dropout = tf.nn.dropout(hidden, keep_proba)
    return tf.matmul(dropout, w4) + b4
  
  # Training computation.
  logits = model(tf_train_dataset, dropout=True)
  cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  l2_loss = (layer1_weight_decay * tf.nn.l2_loss(layer1_weights) 
             + layer2_weight_decay * tf.nn.l2_loss(layer2_weights)
             + layer3_weight_decay * tf.nn.l2_loss(layer3_weights)
             + layer4_weight_decay * tf.nn.l2_loss(layer4_weights))
    
  loss = cross_entropy + l2_loss
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.02).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, averaged=True))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, averaged=True))

In [79]:
# 94.1% with 1x1 'inception' layer.  
# 95.2% using l2 loss 0.001, 0.005, 0.005, 0.001

num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, _ = session.run(
      [optimizer, loss, train_prediction, variable_averages_op], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 12.9649
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 500 : 4.80742
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 1000 : 4.50034
Minibatch accuracy: 75.0%
Validation accuracy: 82.5%
Minibatch loss at step 1500 : 3.62252
Minibatch accuracy: 93.8%
Validation accuracy: 84.3%
Minibatch loss at step 2000 : 3.05151
Minibatch accuracy: 100.0%
Validation accuracy: 85.0%
Minibatch loss at step 2500 : 3.06884
Minibatch accuracy: 87.5%
Validation accuracy: 85.3%
Minibatch loss at step 3000 : 3.0701
Minibatch accuracy: 81.2%
Validation accuracy: 85.8%
Minibatch loss at step 3500 : 2.28538
Minibatch accuracy: 100.0%
Validation accuracy: 86.2%
Minibatch loss at step 4000 : 2.64975
Minibatch accuracy: 93.8%
Validation accuracy: 86.5%
Minibatch loss at step 4500 : 2.62689
Minibatch accuracy: 75.0%
Validation accuracy: 86.7%
Minibatch loss at step 5000 : 1.96074
Minibatch accuracy: 93.8%
Validation accura